In [ ]:
#install pyspark library
!pip install pyspark

In [ ]:
#import pyspark
import pyspark

In [ ]:
#import sparksession 
from pyspark.sql import SparkSession

In [ ]:
#creating a sparksession object and providing appName 
spark=SparkSession.builder.appName("partitions").getOrCreate()

In [ ]:
#To create dataframe form External datasets
%time AirlineDF = spark.read.option("header", "true").csv("/Users/user/Desktop/partition/data/*")

In [ ]:
AirlineDF.show()

In [ ]:
#To check what data in created dataframe
AirlineDF.take(1)

In [ ]:
# calculate total rows in dataframe
AirlineDF.count()

In [ ]:
#calculate total columns in dataframe
len(AirlineDF.columns)

# Partition on dataframe

# reduce and increase no. of partition using coalesce() and repartition()

# here we are partition data according to size or no. of rows

In [ ]:
#to check how many partation in current AirlineDF dataframe
AirlineDF.rdd.getNumPartitions()

In [ ]:
#write dataframe in csv file and cross check no of partition
%time AirlineDF.write.mode("overwrite").csv("flight/airdata", header=True)

In [ ]:
# reduce the no of partition from 14 to 5 here we are using coalesce() function
#using coalesce() you can only reduce the no. of partion not increase no. of partition
%time AirlineDF1 = AirlineDF.coalesce(5)

In [ ]:
#to check how many partation in current AirlineDF1 dataframe
AirlineDF1.rdd.getNumPartitions()

In [ ]:
#write dataframe in csv file and cross check no of partition
%time AirlineDF1.write.mode("overwrite").csv("flight/airline1", header=True)

In [ ]:
#To increase the no of partition from 5 to 10 here we are using repartition() function
%time AirlineDF2 = AirlineDF1.repartition(10)

In [ ]:
#to check how many partation in current AirlineDF2 dataframe
AirlineDF2.rdd.getNumPartitions()

In [ ]:
#write dataframe in csv file and cross check no of partition
%time AirlineDF2.write.mode("overwrite").csv("flight/airline2", header=True)

# partition dataframe  by perticular column

# here we are using partitionBy() 

In [ ]:
#partition data by one column
%time AirlineDF.write.option("header",True) \
        .partitionBy("Year") \
        .mode("overwrite") \
        .csv("flight/airline3")

In [ ]:
#partition data by multiple columns
%time AirlineDF.write.option("header",True) \
        .partitionBy("Year","UniqueCarrier") \
        .mode("overwrite") \
        .csv("flight/airline4")

# Read partition data in pyspark with specific columns

In [ ]:
#Year all and Reporting_Airline AA
%time AirlineDF4 = spark.read.option("basePath", "flight/airline4").csv("flight/airline4/Year=2015/UniqueCarrier=*")

In [ ]:
#To check
%time AirlineDF4.select("Year","UniqueCarrier").show()

In [ ]:
#Year 2015 and Reporting_Airline all
%time AirlineDF5 = spark.read.option("basePath", "flight/airline4").csv("flight/airline4/Year=*/UniqueCarrier=*")

In [ ]:
%time AirlineDF5.select("Year","UniqueCarrier").show()

In [ ]:
#To check
%time AirlineDF5.select("Year","UniqueCarrier").show()

In [ ]:
#perform on partition data
%time AirlineDF5.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

In [ ]:
#without partition
%time AirlineDF.select("Year","UniqueCarrier").groupby("UniqueCarrier").count().show()

# using repartition and partitionBy together

In [ ]:
#Use repartition() and partitionBy() together
%time AirlineDF.repartition(3) \
        .write.option("header",True) \
        .partitionBy("Year") \
        .mode("overwrite") \
        .csv("flight/airline5")

#to check what happen when both are applying together

# partitionBy() control number of partitions


In [ ]:
%time AirlineDF.write.option("header",True) \
        .option("maxRecordsPerFile", 2) \
        .partitionBy("Year") \
        .mode("overwrite") \
        .csv("flight/airline6")
#to check what happen when applying maxRecordsPerFile is 2

# with partation

In [ ]:
# calculate total no of flights and  group data by Carrier
%time  AirlineDF5.select('Year','UniqueCarrier') \
.filter((AirlineDF5.Year >= 2005)).groupBy('UniqueCarrier') \
.count().withColumnRenamed("count", "Total Flights").orderBy("UniqueCarrier").show()

# without partation

In [ ]:
# calculate total no of flights and  group data by Carrier
%time AirlineDF.select('Year','UniqueCarrier') \
.filter((AirlineDF.Year >= 2005)).groupBy('UniqueCarrier') \
.count().withColumnRenamed("count", "Total Flights").orderBy("UniqueCarrier").show()